# to do：
- 回顾之前写的划分MFD的notebook，添加定量分析的函数（CHindex（在写好的部分论文里），NSk，TV）。
- 选出数据，喂到这个notebook里
- adaptation loss前面的lambda用两个类（v_class的均值）之间的KL散度
- boundary adjustment的时候考虑双向，尽量提供错位的数据，即两个方向可能属于不同类（仅用于画图）

- 寻找reinforcement learning用于CAV的算法

In [42]:
import time
import math
import tensorflow as tf
import keras 
import numpy as np
import pandas as pd
import ipdb

In [ ]:
from keras.models import load_model,Model
from keras.engine.topology import Layer
from keras import backend as K

In [2]:
# 定义融合层，将深度学习算法与历史均值算法融合
class Merge_Layer(Layer):
    def __init__(self, **kwargs):
        super(Merge_Layer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.para1 = self.add_weight(shape=(input_shape[0][1], input_shape[0][2]),
                                     initializer='uniform', trainable=True,
                                     name='para1')
        self.para2 = self.add_weight(shape=(input_shape[1][1], input_shape[1][2]),
                                     initializer='uniform', trainable=True,
                                     name='para2')
        super(Merge_Layer, self).build(input_shape)

    def call(self, inputs):
        mat1 = inputs[0]
        mat2 = inputs[1]
        output = mat1 * self.para1 + mat2 * self.para2
        # output = mat1 * 0.1 + mat2 * 0.9
        return output

    def compute_output_shape(self, input_shape):
        return input_shape[0]

In [43]:
#定义精度评价指标。为防止0值附近相对误差过大而导致的异常，定义mask层。
def mape_loss_func(preds, labels):
    mask = labels > 5
    return np.mean(np.fabs(labels[mask]-preds[mask])/labels[mask])

def smape_loss_func(preds, labels):
    mask= labels > 5
    return np.mean(2*np.fabs(labels[mask]-preds[mask])/(np.fabs(labels[mask])+np.fabs(preds[mask])))

def mae_loss_func(preds, labels):
    mask= labels > 5
    return np.fabs((labels[mask]-preds[mask])).mean()

def eliminate_nan(b):
    a = np.array(b)
    c = a[~np.isnan(a)]
    return c

## 把速度矩阵分类

In [55]:
randseed = 3

v = pd.read_csv('./data/v_20_aggragated.csv')
v = v.rename(columns={'Unnamed: 0': 'id'})
det_with_class = pd.read_csv('./res/%i_id_402_withclass.csv'%randseed, index_col=0)

v['class_i'] = ''
for i in range(len(v)):
    v.loc[i, 'class_i'] = det_with_class[det_with_class['id']==v.loc[i, 'id']].iloc[0, 5]  # 5 stands for 'class_i'

num_class = det_with_class['class_i'].drop_duplicates().size

v_class = []
for i in range(num_class):
    v_class.append(v[v['class_i']==i])

print('There are %i class(es)'%num_class)

There are 8 class(es)


## 制作 nearest_road_id.csv 和speed.csv

In [56]:
dist_mat = pd.read_csv('./data/dist_mat.csv', index_col=0)
id_info = pd.read_csv('./data/id2000.csv', index_col=0)
dist_mat.index = id_info['id2']
dist_mat.columns = id_info['id2']
for i in range(len(dist_mat)):
    for j in range(len(dist_mat)):
        if i==j:
            dist_mat.iloc[i, j] = 0

near_id = pd.DataFrame(np.argsort(np.array(dist_mat)), index = id_info['id2'], columns = id_info['id2'])

## 以上做好了near_road矩阵，接下来做flow/speed矩阵

In [57]:
def get_node(det, seg):
    # det is one single detector id
    # node is one single node id
    
    # seg = pd.read_csv('./data/segement.csv', header=None)
    try:
        node_info = seg[seg[6]==det]
        node = node_info.iloc[0, 0]
    except:
        node_info = seg[seg[7]==det]
        node = node_info.iloc[0, 0]
        
    return node

In [58]:
def get_class_with_node(seg, v_class):
    det_list_class = np.array([])
    try:
        v_class.insert(1, 'id2', '')  # id2 mean node id
    except:
        v_class['id2'] = ''
        
    for i in range(len(v_class)):
        det_list_class = np.append(det_list_class, v_class.iloc[i, 0])
        v_class.iloc[i, 1] = get_node(v_class.iloc[i, 0], seg)
    
    return det_list_class, v_class

In [59]:
def rds_mat(old_dist_mat, det_ids):
    # get a matrix that contains n raods that have specified node id s
    node_ids = np.array([])
    for i in det_ids:
        node_ids = np.append(node_ids, get_node(i, seg))
        
    new_dist_mat = old_dist_mat.loc[node_ids, node_ids]
    old_dist_mat = np.array(old_dist_mat)
    new_near_id_mat = np.argsort(new_dist_mat)
    return new_near_id_mat

In [60]:
seg = pd.read_csv('./data/segement.csv', header=None)
num_dets = 30

det_list_class = []
for i in range(num_class):
    det_list_class_temp, v_class_temp = get_class_with_node(seg, v_class[i])
    det_list_class.append(det_list_class_temp)
    v_class_temp = v_class_temp[v_class_temp['id'].isin(det_list_class_temp[:num_dets])]
    v_class[i] = v_class_temp

In [61]:
near_road = []
for i in range(num_class):
    near_road.append(rds_mat(dist_mat, det_list_class[i][:num_dets]))

## evaluation of 2 datasets

In [62]:
v_class[0].iloc[:, 2:-1].T.mean().T.std()

13.606684450518879

In [63]:
def get_NSk(set1, set2):
    # designated for v_class1 and 2
    set1_v_mean = set1.iloc[:, 2:-1].T.mean().T
    set2_v_mean = set2.iloc[:, 2:-1].T.mean().T
    
    var1 = set1_v_mean.std()**2
    var2 = set2_v_mean.std()**2
    
    u1 = set1_v_mean.mean()
    u2 = set2_v_mean.mean()
    
    return 2*var1 / (var1 + var2 + (u1 - u2)**2)

In [64]:
get_NSk(v_class[0], v_class[1])

0.8972017877887051

In [65]:
NSk_set = np.array([])
for i in range(num_class):
    for j in range(num_class):
        if i!=j:
            NSk = get_NSk(v_class[i], v_class[j])
            NSk_set = np.append(NSk_set, NSk)

print(NSk_set.mean())

0.9261353004471786


# 源代码如下

In [16]:
# near_road = np.array(pd.read_csv('./data/network/2small_network_nearest_road_id.csv',header = 0))
# flow = np.array(pd.read_csv('./data/network/2small_network_speed.csv', header= 0)) #注意header=0 or None
near_road = np.array(near_road1)
flow = np.array(v_class1.iloc[:, 2:-1])

# 利用滑动窗口的方式，重构数据为(n，最近路段数，输入时间窗，总路段数)的形式

time3 = time.time()

k = 5 # 参数k为需考虑的最近路段数
t_p = 24 # 参数t_p为总时间序列长度（天）
t_input = 12 #参数t_input为输入时间窗(5min颗粒度)
t_pre = 3 #参数t_pre为预测时间窗(5min颗粒度)
num_links = 30 #参数num_links为总路段数


image = []
for i in range(np.shape(near_road)[0]):
    road_id = []
    for j in range(k):
        road_id.append(near_road[i][j])
    image.append(flow[road_id, :])
image1 = np.reshape(image, [-1, k, len(flow[0,:])])
image2 = np.transpose(image1,(1,2,0))
image3 = []
label = []
day = []

for i in range(1,t_p):
    for j in range(180-t_input-t_pre):
        image3.append(image2[:, i*180+j:i*180+j+t_input, :][:])
        label.append(flow[:, i*180+j+t_input:i*180+j+t_input+t_pre][:])
        day.append(flow[:, (i-1)*180+j+t_input:(i-1)*180+j+t_input+t_pre][:])
        
# ipdb.set_trace()

image3 = np.asarray(image3)
label = np.asarray(label)
day =  np.asarray(day)

print(np.shape(image3))
print(np.shape(label))
print(np.shape(day))

#划分前90%数据为训练集，最后10%数据为测试集
image_train_source = image3[:np.shape(image3)[0]*1//10]
image_test_source = image3[np.shape(image3)[0]*1//10:]
label_train_source = label[:np.shape(label)[0]*1//10]
label_test_source = label[np.shape(label)[0]*1//10:]

day_train_source = day[:np.shape(day)[0]*1//10]
day_test_source = day[np.shape(day)[0]*1//10:]


time4 = time.time()
print('input done %g' % (time4-time3))

(3795, 5, 12, 30)
(3795, 30, 3)
(3795, 30, 3)
input done 0.0368743


In [17]:
# near_road = np.array(pd.read_csv('./data/transfer_learning_traffic_data/small_network_nearest_road_id.csv',header = 0))
# flow = np.array(pd.read_csv('./data/transfer_learning_traffic_data/small_network_speed.csv', header= 0)) #注意header=0 or None
near_road = np.array(near_road2)
flow = np.array(v_class2.iloc[:, 2:-1])

# 利用滑动窗口的方式，重构数据为(n，最近路段数，输入时间窗，总路段数)的形式

time3 = time.time()

k = 5 # 参数k为需考虑的最近路段数
t_p = 24 # 参数t_p为总时间序列长度（天）
t_input = 12 #参数t_input为输入时间窗(5min颗粒度)
t_pre = 3 #参数t_pre为预测时间窗(5min颗粒度)
num_links = 30 #参数num_links为总路段数


image = []
for i in range(np.shape(near_road)[0]):
    road_id = []
    for j in range(k):
        road_id.append(near_road[i][j])
    image.append(flow[road_id, :])
image1 = np.reshape(image, [-1, k, len(flow[0,:])])
image2 = np.transpose(image1,(1,2,0))
image3 = []
label = []
day = []

for i in range(1,t_p):
    for j in range(180-t_input-t_pre):
        image3.append(image2[:, i*180+j:i*180+j+t_input, :][:])
        label.append(flow[:, i*180+j+t_input:i*180+j+t_input+t_pre][:])
        day.append(flow[:, (i-1)*180+j+t_input:(i-1)*180+j+t_input+t_pre][:])

image3 = np.asarray(image3)
label = np.asarray(label)
day =  np.asarray(day)

print(np.shape(image3))
print(np.shape(label))
print(np.shape(day))

#划分前80%数据为训练集，最后20%数据为测试集
image_train_target = image3[:np.shape(image3)[0]*1//10]
image_test_target = image3[np.shape(image3)[0]*1//10:]
label_train_target = label[:np.shape(label)[0]*1//10]
label_test_target = label[np.shape(label)[0]*1//10:]

day_train_target = day[:np.shape(day)[0]*1//10]
day_test_target = day[np.shape(day)[0]*1//10:]


time4 = time.time()
print('input done %g' % (time4-time3))

(3795, 5, 12, 30)
(3795, 30, 3)
(3795, 30, 3)
input done 0.0420151


In [18]:
#模型构建
input_data = keras.Input(shape=(k,t_input,num_links), name='input_data')
input_HA = keras.Input(shape=(num_links, t_pre), name='input_HA')

x = keras.layers.BatchNormalization(input_shape =(k,t_input,num_links))(input_data)

x = keras.layers.Conv2D(
                           filters = num_links,
                           kernel_size = 3,
                           strides = 1,
                           padding="SAME",
                           activation='relu')(x)

x = keras.layers.AveragePooling2D(pool_size = (2,2),
                                strides = 1,
                                padding = "SAME",
                                )(x)

x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(
                       filters = num_links,
                       kernel_size = 3,
                       strides = 1,
                       padding="SAME",
                       activation='relu')(x)

x = keras.layers.AveragePooling2D(pool_size = (2,2),
                                strides = 1,
                                padding = "SAME",
                                )(x)
x = keras.layers.Flatten()(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(num_links*t_pre, activation='relu')(x)

output = keras.layers.Reshape((num_links,t_pre))(x)

output_final = Merge_Layer()([output, input_HA])

# construct model
finish_model = keras.models.Model([input_data,input_HA], [output_final])

finish_model.summary()









Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_data (InputLayer)         (None, 5, 12, 30)    0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 5, 12, 30)    120         input_data[0][0]                 
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 5, 12, 30)    8130        batch_normalization_1[0][0]      
__________________________________________________________________________________________________
average_pooling2d_1 (AveragePoo (None, 5, 12, 30)    0           conv2d_1[0][0]       

In [19]:
#参数加载
finish_model.load_weights('./model/source.h5')


In [24]:
#模型预测
model_pre = finish_model.predict([image_test_target,day_test_target])


In [25]:
#预测结果存储
# model_pre = np.reshape(model_pre,[103, 6])
# model_pre1 = pd.DataFrame(model_pre)
# model_pre1.to_csv('预测值.csv', index = False)

In [26]:
#transfer without FT 预测精度计算

mape_mean = mape_loss_func(model_pre, label_test_source)
smape_mean = smape_loss_func(model_pre, label_test_source)
mae_mean = mae_loss_func(model_pre, label_test_source)

print('mape = ' + str(mape_mean) + '\n' + 'smape = ' + str(smape_mean) + '\n' + 'mae = ' + str(mae_mean))

mape = 0.7802823984459177
smape = 1.3187246744990813
mae = 48.69012338609012


In [27]:

middle = Model(inputs=[input_data, input_HA],outputs=finish_model.get_layer('dense_1').output)

In [28]:
middle_result_source = middle.predict([image_train_source, day_train_source])
middle_result_target = middle.predict([image_train_target, day_train_target])

In [29]:
from sklearn import metrics
def mmd (x, y):
    return metrics.mean_squared_error(x,y)

In [30]:
mmd (middle_result_source, middle_result_target)

65.25655

In [31]:
def kl_divergence(set1, set2):
    set1_v_mean = np.array(set1.iloc[:, 2:-1].T.mean().T)
    set2_v_mean = np.array(set2.iloc[:, 2:-1].T.mean().T)
    return np.sum(np.where(set1_v_mean != 0, set1_v_mean * np.log(set1_v_mean / set2_v_mean), 0))

In [32]:
lamb = kl_divergence(v_class1, v_class2)

loss1 = K.mean(K.square(output_final - label_train_target), axis=-1) 
loss2 = lamb * mmd (middle_result_source, middle_result_target)
overall_loss = loss1 + loss2

In [33]:
def new_loss(output_final, label_train_target):
    middle = Model(inputs=[input_data, input_HA],outputs=finish_model.get_layer('dense_1').output)
    middle_result_source = middle.predict([image_train_source, day_train_source])
    middle_result_target = middle.predict([image_train_target, day_train_target])

    loss1 = K.mean(K.square(output_final - label_train_target), axis=-1) 
    loss2 = 0.001 * mmd (middle_result_source, middle_result_target)
    overall_loss = loss1 + loss2
    return overall_loss


In [34]:
finish_model.compile(optimizer='adam',loss=new_loss)

In [35]:
finish_model.fit([image_train_target, day_train_target], label_train_target, epochs=100, batch_size=462,
validation_data=([image_test_target,day_test_target], label_test_target))

Train on 379 samples, validate on 3416 samples
Epoch 1/100
379/379 [==============================] - 1s 2ms/step - loss: 1343.0009 - val_loss: 1351.6359
Epoch 2/100
379/379 [==============================] - 0s 258us/step - loss: 1274.7327 - val_loss: 1347.7678
Epoch 3/100
379/379 [==============================] - 0s 287us/step - loss: 1224.3678 - val_loss: 1333.5440
Epoch 4/100
379/379 [==============================] - 0s 260us/step - loss: 1172.6211 - val_loss: 1310.1892
Epoch 5/100
379/379 [==============================] - 0s 250us/step - loss: 1133.0148 - val_loss: 1283.4592
Epoch 6/100
379/379 [==============================] - 0s 245us/step - loss: 1095.9395 - val_loss: 1254.8227
Epoch 7/100
379/379 [==============================] - 0s 268us/step - loss: 1058.7548 - val_loss: 1229.4635
Epoch 8/100
379/379 [==============================] - 0s 242us/step - loss: 1022.5051 - val_loss: 1210.1482
Epoch 9/100
379/379 [==============================] - 0s 266us/step - loss: 986.47

Epoch 77/100
379/379 [==============================] - 0s 242us/step - loss: 263.9236 - val_loss: 195.7713
Epoch 78/100
379/379 [==============================] - 0s 255us/step - loss: 260.5092 - val_loss: 202.6742
Epoch 79/100
379/379 [==============================] - 0s 245us/step - loss: 256.7712 - val_loss: 208.5512
Epoch 80/100
379/379 [==============================] - 0s 245us/step - loss: 254.0020 - val_loss: 212.1134
Epoch 81/100
379/379 [==============================] - 0s 245us/step - loss: 251.8787 - val_loss: 213.5305
Epoch 82/100
379/379 [==============================] - 0s 237us/step - loss: 248.5002 - val_loss: 212.5414
Epoch 83/100
379/379 [==============================] - 0s 258us/step - loss: 243.4228 - val_loss: 210.0705
Epoch 84/100
379/379 [==============================] - 0s 245us/step - loss: 242.0824 - val_loss: 205.9733
Epoch 85/100
379/379 [==============================] - 0s 253us/step - loss: 240.6344 - val_loss: 201.0364
Epoch 86/100
379/379 [======

In [36]:
model_pre = finish_model.predict([image_test_target,day_test_target])

In [37]:
#transfer with DAN 预测精度计算

mape_mean = mape_loss_func(model_pre, label_test_target)
smape_mean = smape_loss_func(model_pre, label_test_target)
mae_mean = mae_loss_func(model_pre, label_test_target)

print('mape = ' + str(mape_mean) + '\n' + 'smape = ' + str(smape_mean) + '\n' + 'mae = ' + str(mae_mean))

mape = 0.2113231710602907
smape = 0.1623835174380695
mae = 8.379577988135262


In [38]:
mape_list = []
for i in range(num_links):
    a1 = mape_loss_func(model_pre[:,i,:], label_test_target[:,i,:])
    mape_list.append(a1)

mape_pd = pd.Series(mape_list)
mape_pd.sort_values()

23    0.059805
12    0.067795
16    0.075069
6     0.089917
25    0.094435
29    0.105310
4     0.106422
8     0.113243
13    0.114430
15    0.125810
19    0.131992
9     0.132237
27    0.132476
28    0.146909
1     0.159441
22    0.163477
10    0.173699
11    0.188523
14    0.202844
18    0.208890
7     0.208979
3     0.237249
24    0.287715
21    0.291012
20    0.322721
2     0.338975
0     0.385404
26    0.439839
17    0.582887
5     0.652423
dtype: float64